# Modelization

In [15]:
# Set code path
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("Model.ipynb"), '..', 'code')))

In [16]:
from impute_and_drop import impute_missing, drop_data
from one_hot_encoding import one_hot_encoding
#from preprocess import get_preprocessed_data
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [17]:
#data = get_preprocessed_data(data_file_name="cis2080.csv", labels_file_name="descriptive_var_names.json")
from utils import *
data = load_my_data()

## Subset data
Select only those atributes whose metadata is considered complete.

In [18]:
with open("../metadata/descriptive_var_names.json") as f:
   var_names = json.load(f)

selected_vars = ["UBE"]
for key, value in var_names.items():
    if (value["description"] != "incomplete"):
        selected_vars.append(value["name"])

subdata = data[selected_vars].drop(columns=["day", "month", "year"])

## Missing values imputation
### Exploration

In [22]:
data = impute_missing(subdata)
data = drop_data(data)

## One hot encoding

In [23]:
X, y = one_hot_encoding(data, "UBE")

## RandomForestClassifier

In [24]:
# Split the dataset into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [25]:
RFC = RandomForestRegressor(n_estimators=5, max_depth=4, random_state=1)


In [26]:
RFC.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, n_estimators=5, random_state=1)

In [27]:
# Make predictions on the test data
y_pred = RFC.predict(X_test)

# Evaluate the performance of the classifier on the test data
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: {:.2f}".format(rmse))

RMSE: 7.05


In [31]:
data.UBE.quantile([0.1, 0.9])

0.1     0.0
0.9    14.0
Name: UBE, dtype: float64

Given these quantiles, the RMSE indicates that the prediction is awful. Nevertheless, there are plenty of aspects to improve now that a basic model is running.